In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/DATA')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings(action='ignore')


#### Data loading

In [4]:
train=pd.read_csv('./train_features.csv')
train_labels=pd.read_csv('./train_labels.csv')
test=pd.read_csv('./0616Lee.csv')

submission=pd.read_csv('./sample_submission_0616Lee.csv')

pd.options.display.max_columns=50

In [5]:
test = test.drop('exercise',axis=1)
test.rename(columns = {'Time' : 'time'}, inplace = True)
test

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,2021-06-15 10:43:34:05505,0.886565,-0.693919,0.003907,-30.66,-40.60,-43.75
1,0,2021-06-15 10:43:34:4155,0.962501,0.129896,-0.198751,-1.05,17.36,27.72
2,0,2021-06-15 10:43:34:43266,0.971779,0.098155,-0.140151,24.08,25.55,39.20
3,0,2021-06-15 10:43:34:45299,1.077747,-0.172625,-0.000488,0.56,40.81,43.19
4,0,2021-06-15 10:43:34:47443,0.991801,-0.340367,-0.140884,-33.60,32.90,42.98
...,...,...,...,...,...,...,...,...
45595,75,2021-06-15 12:54:27:86189,0.833581,-0.556209,-0.130384,-6.72,4.55,1.61
45596,75,2021-06-15 12:54:27:87987,0.825280,-0.566220,-0.110119,-4.41,3.50,1.12
45597,75,2021-06-15 12:54:27:90266,0.804037,-0.575742,-0.105235,-2.10,2.80,0.98
45598,75,2021-06-15 12:54:27:91982,0.808188,-0.582091,-0.107433,-0.98,2.87,0.91


In [6]:
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


#### Feature engneering

#####  가속도, 자이로, (자이로-가속도) 센서값을 에너지로 표현

In [7]:
# 왜 세제곱근일까? 
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/2)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/2)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/2)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/2)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/2)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/2)

###### id별 데이터는 0.02초마다 측정된 값들이기 때문에 이전 시간 대비 변화량 적용

In [8]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [9]:
train_dt=[]
# tqdm => 진행표시바 나오게 함
# np.unique => 고유한 원소의 집합 
for i in tqdm(train['id'].unique()): # id 0~3124 
    # 한 아이디에 600rows가 있음
    temp=train.loc[train['id']==i]
    # acc xyz, gy xyz, 위에서 생성한 energy 컬럼 3개
    for v in train.columns[2:]:
        # 컬럼의 값들만 jerk_signal 함수에 넣은 뒤 반환
        values=jerk_signal(temp[v].values)
        
        # values의 0번째 위치에 0 삽입
        values=np.insert(values,0,0)
        # v+'_dt' 라는 컬럼에 values값 삽입
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)


100%|██████████| 3125/3125 [00:44<00:00, 69.65it/s]


In [10]:
train_dt[0]

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,1.228355,44.010999,43.812559,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,1.315689,46.403958,46.147664,4.080495,-0.980114,-1.699854,44.735403,-429.504677,337.444793,4.366731,119.647963,116.755248
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,1.343234,50.851423,50.667670,0.845632,0.192961,-3.546937,-196.018888,-249.173073,-4.620631,1.377227,222.373226,226.000319
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,1.330969,60.281107,59.933763,-0.575711,-1.762585,1.908626,316.513181,-473.763910,-121.719195,-0.613248,471.484195,463.304675
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,1.333070,64.446693,64.107811,0.389598,2.130453,-0.365665,78.686055,-215.435892,-25.361098,0.105059,208.279312,208.702400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,595,0.985242,-0.326122,-0.354528,-14.903280,20.172339,22.973018,1.096698,34.011624,34.882752,1.177958,-4.242820,-0.324471,-220.368219,-286.214412,340.765651,2.295085,86.679887,95.725074
596,0,596,1.052837,-0.220710,-0.413472,-10.857025,19.786856,23.174597,1.152449,32.348982,33.144299,3.379771,5.270607,-2.947208,202.312749,-19.274131,10.078975,2.787526,-83.132095,-86.922683
597,0,597,1.025643,-0.227845,-0.354516,-2.334243,25.768654,18.932070,1.108846,32.060810,32.543498,-1.359731,-0.356764,2.947780,426.139108,299.089890,-212.126333,-2.180170,-14.408606,-30.040044
598,0,598,1.031553,-0.387862,-0.277857,-9.710746,28.697694,20.631577,1.136549,36.654034,37.397473,0.295512,-8.000850,3.832979,-368.825182,146.451989,84.975330,1.385165,229.661212,242.698772


In [11]:
test_dt=[]
# train과 같은 방식으로 진행
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|██████████| 76/76 [00:00<00:00, 80.71it/s]


##### 가속도, 자이로 센서값들을 푸리에 변환

In [12]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    # fft => Fast Fourier transforms
    # 매개변수로 받은 t_signal을 푸리에 변환 시킨다
    complex_f_signal= fftpack.fft(t_signal)
     
    # 주파수 해석의 목적은 각 주파수 별로 크기 (magnitude)가 어느 정도인지 알아내는 것
    # 따라서 절대값 함수 abs()를 이용해서 크기를 연산할 수 있음
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [13]:
train=pd.concat(train_dt)

In [14]:
# train

In [15]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    # acc값과 gy 값만 사용
    for i in train.columns[2:8]:
        # 위에서 선언한 함수 사용해서 푸리에 변환
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████| 3125/3125 [00:10<00:00, 295.12it/s]


In [16]:
# train

In [16]:
test=pd.concat(test_dt)

In [17]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|██████████| 76/76 [00:00<00:00, 576.73it/s]


##### Standard scaling 적용


In [18]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [19]:
train_s

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,0,558.797337,131.082711,222.252919,1119.161589,2015.703683,709.264425,1.228355,44.010999,43.812559,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,3.233175,15.689279,12.229014,221.599635,361.903330,477.080942,1.315689,46.403958,46.147664,4.080495,-0.980114,-1.699854,44.735403,-429.504677,337.444793,4.366731,119.647963,116.755248
2,0,2,4.832535,8.199566,3.901211,357.200415,430.568986,452.096143,1.343234,50.851423,50.667670,0.845632,0.192961,-3.546937,-196.018888,-249.173073,-4.620631,1.377227,222.373226,226.000319
3,0,3,5.675383,5.330015,2.527445,340.433376,787.558320,467.307109,1.330969,60.281107,59.933763,-0.575711,-1.762585,1.908626,316.513181,-473.763910,-121.719195,-0.613248,471.484195,463.304675
4,0,4,7.415275,7.980024,6.566908,128.188871,1372.095224,715.824074,1.333070,64.446693,64.107811,0.389598,2.130453,-0.365665,78.686055,-215.435892,-25.361098,0.105059,208.279312,208.702400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,11.743654,3.796333,12.513870,715.873677,1124.494889,645.627066,1.013606,132.293402,131.789903,1.484646,0.303666,0.800069,-150.644663,-34.630282,-8.380088,-1.027613,64.000025,64.267455
1874996,3124,596,211.498089,82.888508,86.807874,5515.261695,28917.564390,20218.747027,1.004243,130.931134,130.457574,1.474659,-0.005442,1.775771,-39.061611,110.842743,-16.732496,-0.468141,-68.113373,-66.616447
1874997,3124,597,12.175349,6.200258,2.084554,343.695161,464.375112,78.097163,1.009373,129.739912,129.315545,0.915321,-0.407957,1.744566,113.799702,151.036858,-137.001896,0.256510,-59.561085,-57.101417
1874998,3124,598,19.116783,3.830800,6.938661,791.376179,2724.373764,1131.590078,1.001557,129.726288,129.324215,1.709833,-0.796984,0.479107,211.827245,-18.171144,-44.717652,-0.390820,-0.681232,0.433504


In [20]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

# Standard => 기본 스케일. 평균과 표준편차 사용
# 평균을 제거하고 데이터를 단위 분산으로 조정
# 이상치가 있다면 평균과 표준편차에 영향을 미쳐 변환된 데이터의 확산은 매우 달라지게 된다
# 따라서 이상치가 있는 경우 균형 잡힌 척도를 보장할 수 없다
scaler = StandardScaler()


# 변환만을 생각한다면 fit() , transform()을 함께 사용하지 않고 
# transform()만 사용하면 될텐데 두개 메소드를 함께 사용하는 이유가 있다

# 학습데이터 세트에서 변환을 위한 기반 설정(예를 들어 학습 데이터 세트의 최대값/최소값등)을 
# 먼저 fit()을 통해서 설정한 뒤에 이를 기반으로 학습 데이터의 transform()을 수행하되 
# 학습 데이터에서 설정된 변환을 위한 기반 설정을 그대로 테스트 데이터에도 적용하기 위해서이다
# 참고 https://subinium.github.io/MLwithPython-3-3/
train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [21]:
test_s

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,2021-06-15 10:43:34:05505,28.174976,10.419897,5.217735,-0.085647,0.053592,0.217031,0.127639,-0.095441,-0.095624,0.000027,0.000298,-0.000433,0.000347,0.000373,0.000273,0.000050,0.001067,0.001067
1,0,2021-06-15 10:43:34:4155,0.051227,0.386192,0.619065,-0.065798,-0.036759,0.329390,-0.211443,-0.472347,-0.470884,0.387855,5.006484,-1.518231,1.072311,2.358335,3.990670,-0.726108,-1.465528,-1.459423
2,0,2021-06-15 10:43:34:43266,0.237732,0.063659,-0.025030,0.002916,-0.150036,0.273969,-0.223107,-0.254161,-0.258470,0.047414,-0.192590,0.438448,0.910123,0.333563,0.641238,-0.024929,0.850058,0.827769
3,0,2021-06-15 10:43:34:45299,0.547696,0.284304,0.629582,0.422472,0.309511,0.403600,0.041000,-0.179612,-0.178458,0.541240,-1.645185,1.045567,-0.851143,0.621189,0.223047,0.565647,0.291151,0.312470
4,0,2021-06-15 10:43:34:47443,1.352864,0.283086,0.017130,1.554221,1.016960,1.486389,-0.043416,-0.132640,-0.124045,-0.438929,-1.019041,-1.051919,-1.236340,-0.321427,-0.011452,-0.180731,0.183840,0.212839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45595,75,2021-06-15 12:54:27:86189,1.150294,0.618547,-0.014429,-0.257918,0.203665,0.903406,-0.163028,-0.740427,-0.729478,-0.162088,0.000298,0.211693,0.030757,-0.036649,-0.046627,-0.163430,-0.057310,-0.058424
45596,75,2021-06-15 12:54:27:87987,0.535259,0.357470,0.297271,-0.124754,0.273266,0.903835,-0.172286,-0.768202,-0.757326,-0.042372,-0.060536,0.151347,0.083975,-0.042344,-0.027086,-0.019777,-0.107007,-0.107317
45597,75,2021-06-15 12:54:27:90266,2.918196,1.368807,0.177573,-0.182874,0.095839,1.778981,-0.203511,-0.791290,-0.780994,-0.108465,-0.057568,0.036140,0.083975,-0.028105,-0.007544,-0.066821,-0.088773,-0.091045
45598,75,2021-06-15 12:54:27:91982,1.713706,0.384604,0.620387,-0.019725,0.507979,0.928336,-0.185187,-0.796425,-0.787184,0.021227,-0.038279,-0.016891,0.040894,0.003220,-0.003636,0.039293,-0.018913,-0.023026


##### 모델링

+ CNN, LSTM, CNN+LSTM 등 여러 구조 적용해보다가 CNN에서 Flatten 없이 Global average pooling 한 구조가 가장 성능이 좋아 채택했습니다.


In [22]:
pip install tensorflow_addons

     |████████████████████████████████| 686kB 8.4MB/s 


In [23]:
pip install np_utils

     |████████████████████████████████| 61kB 4.9MB/s 
  Created wheel for np-utils: filename=np_utils-0.5.12.1-cp37-none-any.whl size=57133 sha256=371ad33d61f839bf1c50aa26aeb6bc547c1f337a2920239b9b5f37cb1b2c69aa
  Stored in directory: /root/.cache/pip/wheels/92/4b/81/206efd0d01330a96f3aebe5021d2d5f0b264b7ade827c306ef
Successfully built np-utils


In [24]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, LSTM,Bidirectional,Dropout
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras import backend as K 
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from sklearn.model_selection import KFold,StratifiedKFold
from numpy.random import seed
import keras

In [25]:
X=np.array(train_sc.iloc[:,2:]).reshape(3125, 600, -1)
# 아이디 별로 600개의 데이터가 있고, 3125개의 아이디가 있다(0~3124)
X.shape

(3125, 600, 18)

In [26]:
test_sc

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_Energy,gy_Energy,gy_acc_Energy,acc_x_dt,acc_y_dt,acc_z_dt,gy_x_dt,gy_y_dt,gy_z_dt,acc_Energy_dt,gy_Energy_dt,gy_acc_Energy_dt
0,0,2021-06-15 10:43:34:05505,28.174976,10.419897,5.217735,-0.085647,0.053592,0.217031,0.127639,-0.095441,-0.095624,0.000027,0.000298,-0.000433,0.000347,0.000373,0.000273,0.000050,0.001067,0.001067
1,0,2021-06-15 10:43:34:4155,0.051227,0.386192,0.619065,-0.065798,-0.036759,0.329390,-0.211443,-0.472347,-0.470884,0.387855,5.006484,-1.518231,1.072311,2.358335,3.990670,-0.726108,-1.465528,-1.459423
2,0,2021-06-15 10:43:34:43266,0.237732,0.063659,-0.025030,0.002916,-0.150036,0.273969,-0.223107,-0.254161,-0.258470,0.047414,-0.192590,0.438448,0.910123,0.333563,0.641238,-0.024929,0.850058,0.827769
3,0,2021-06-15 10:43:34:45299,0.547696,0.284304,0.629582,0.422472,0.309511,0.403600,0.041000,-0.179612,-0.178458,0.541240,-1.645185,1.045567,-0.851143,0.621189,0.223047,0.565647,0.291151,0.312470
4,0,2021-06-15 10:43:34:47443,1.352864,0.283086,0.017130,1.554221,1.016960,1.486389,-0.043416,-0.132640,-0.124045,-0.438929,-1.019041,-1.051919,-1.236340,-0.321427,-0.011452,-0.180731,0.183840,0.212839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45595,75,2021-06-15 12:54:27:86189,1.150294,0.618547,-0.014429,-0.257918,0.203665,0.903406,-0.163028,-0.740427,-0.729478,-0.162088,0.000298,0.211693,0.030757,-0.036649,-0.046627,-0.163430,-0.057310,-0.058424
45596,75,2021-06-15 12:54:27:87987,0.535259,0.357470,0.297271,-0.124754,0.273266,0.903835,-0.172286,-0.768202,-0.757326,-0.042372,-0.060536,0.151347,0.083975,-0.042344,-0.027086,-0.019777,-0.107007,-0.107317
45597,75,2021-06-15 12:54:27:90266,2.918196,1.368807,0.177573,-0.182874,0.095839,1.778981,-0.203511,-0.791290,-0.780994,-0.108465,-0.057568,0.036140,0.083975,-0.028105,-0.007544,-0.066821,-0.088773,-0.091045
45598,75,2021-06-15 12:54:27:91982,1.713706,0.384604,0.620387,-0.019725,0.507979,0.928336,-0.185187,-0.796425,-0.787184,0.021227,-0.038279,-0.016891,0.040894,0.003220,-0.003636,0.039293,-0.018913,-0.023026


In [27]:
test_x=np.array(test_sc.iloc[:,2:]).reshape(76, 600, -1)
# reshape()의 ‘-1’이 의미하는 바는, 변경된 배열의 ‘-1’ 
# 위치의 차원은 “원래 배열의 길이와 남은 차원으로 부터 추정”
# => 차원을 따로 지정하지않고 추정하겠다라는 의미인듯
test_x.shape

(76, 600, 18)

In [28]:
train_labels

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [29]:
y = train_labels['label'].values
y = tf.keras.utils.to_categorical(train_labels['label']) 
print(y)
print("y.shape => ",y.shape)
y[0]

# y[0] 출력해보면 알겠지만 label 번째 인덱스 값만 1이다(0부터 시작했을 경우)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
y.shape =>  (3125, 61)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

##### 모델 구조 

In [30]:
# seed를 지정하면 계속 똑같은 값만 나온다
seed(1)
np.random.rand(10)

array([4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
       1.46755891e-01, 9.23385948e-02, 1.86260211e-01, 3.45560727e-01,
       3.96767474e-01, 5.38816734e-01])

In [31]:
def cnn_model(input_shape, classes):
    # random.seed를 설정해주면 같은 랜덤셋이 나온다했는데..
    # 왜 자꾸 seed를 지정해주는지는 모르겠다 
    seed(2021)
    tf.random.set_seed(2021)
    
    # keras input은 데이터의 입력 모양을 모델에 알려주는 역할
    input_layer = keras.layers.Input(input_shape)
    # Conv1D => 1D 컨볼루션 레이어
    # Layers 부분 너무 어렵다 ㅋ 이해 불가
    conv1 = keras.layers.Conv1D(filters=128, kernel_size=9, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)
    conv1 = keras.layers.Dropout(rate=0.3)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=6, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    conv2 = keras.layers.Dropout(rate=0.4)(conv2)
    
    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    conv3 = keras.layers.Dropout(rate=0.5)(conv3)
    
    gap = keras.layers.GlobalAveragePooling1D()(conv3)
    
    output_layer = keras.layers.Dense(classes, activation='softmax')(gap)
    
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
        metrics=['accuracy'])
    
    return model

##### 10-fold StratifiedKFold

In [32]:
os.getcwd()

'/content/drive/My Drive/DATA'

In [33]:
# 과적합을 방지하기 위한 교차검증
# StratifiedKFold => target에 속성값 개수를 동일하게 가져감으로써 기존의 kfold와 같은 데이터 몰림현상 방지
skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)

# ReduceLROnPlateau => keras의 콜백함수
# 학습률이 개선되지 않을 때 학습률을 동적으로 조정하여 개선하는 효과를 기대할 수 있음
# patience => epoch 4 동안 개선되지 않으면 callback 호출
# verbos => 1일 경우, EarlyStopping이 적용될 때, 화면에 적용되었다고 나타냄, 0일 경우, 화면에 나타냄 없이 종료
# factor => callback 호출 시 학습률을 0.5로 줄인다(1/2)
reLR = ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 

# EarlyStopping => 더 이상 개선의 여지가 없을 때 학습을 종료시키는 콜백함수
# monitor = 'val_loss' => 검증 손실을 기준으로 callback이 호출
# patience => epoch 8 동안 개선되지 않으면 callback 호출
# mode => 관찰 항목이 val_loss인 경우 감소되는 것이 멈출 때 종료되야 하므로 'min'으로 설정
es =EarlyStopping(monitor='val_loss', patience=8, mode='min')

accuracy = []
losss=[]
models=[]

for i, (train, validation) in enumerate(skf.split(X, y.argmax(1))) :
    # ModelCheckpoint => 모델을 저장할 때 사용하는 콜백 함수
    # filepath => 모델을 저장할 경로
    # save_best_only => True 인 경우, monitor 되고 있는 값을 기준으로 가장 좋은 값으로 모델이 저장
    #                => False인 경우, 매 epoch마다 모델이 filepath{epoch}으로 저장 ex) model0,model1,model2..
    # verbose, monitor, mode => 위랑 일맥상통
    # save_weights_only => True인 경우, 모델의 weights만 저장
    #                   -> False인 경우, 모델 레이어 및 weights 모두 저장
    mc = ModelCheckpoint(f'./model_kf/cv_study{i + 1}.h5',save_best_only=True, verbose=0, monitor = 'val_loss', mode = 'min', save_weights_only=True)
    print("-" * 20 +"Fold_"+str(i+1)+ "-" * 20)
    model = cnn_model((600,18),61)
    history = model.fit(X[train], y[train], epochs = 100, validation_data= (X[validation], y[validation]), 
                        verbose=1,batch_size=64,callbacks=[es,mc,reLR])
    model.load_weights(f'./model_kf/cv_study{i + 1}.h5')
    
    # model.evaluate => 테스트 데이터를 통해 학습한 모델에 대한 정확도를 평가
    # 첫번째 인자 = 테스트 데이터에 해당
    # 두번째 인자 = 지도 학습에서 레이블 테스트 데이터에 해당
    # batch_size = 배치 크기
    k_accuracy = '%.4f' % (model.evaluate(X[validation], y[validation])[1])
    k_loss = '%.4f' % (model.evaluate(X[validation], y[validation])[0])
    
    accuracy.append(k_accuracy)
    losss.append(k_loss)
    models.append(model)

print('\nK-fold cross validation Auc: {}'.format(accuracy))
print('\nK-fold cross validation loss: {}'.format(losss))

--------------------Fold_1--------------------
Epoch 1/100
44/44 [==============================] - 48s 65ms/step - loss: 3.4393 - accuracy: 0.3483 - val_loss: 3.2258 - val_accuracy: 0.2045
Epoch 2/100
44/44 [==============================] - 2s 45ms/step - loss: 1.9799 - accuracy: 0.5518 - val_loss: 2.0592 - val_accuracy: 0.5208
Epoch 3/100
44/44 [==============================] - 2s 45ms/step - loss: 1.7910 - accuracy: 0.5596 - val_loss: 2.1110 - val_accuracy: 0.5463
Epoch 4/100
44/44 [==============================] - 2s 45ms/step - loss: 1.5633 - accuracy: 0.6149 - val_loss: 1.9055 - val_accuracy: 0.5623
Epoch 5/100
44/44 [==============================] - 2s 45ms/step - loss: 1.3805 - accuracy: 0.6512 - val_loss: 1.5534 - val_accuracy: 0.6070
Epoch 6/100
44/44 [==============================] - 2s 45ms/step - loss: 1.2710 - accuracy: 0.6785 - val_loss: 1.4004 - val_accuracy: 0.6198
Epoch 7/100
44/44 [==============================] - 2s 45ms/step - loss: 1.1608 - accuracy: 0.6915 

##### 성능 확인 및 제출

In [39]:
print(accuracy)
print()
print(losss)

['0.8403', '0.8435', '0.8690', '0.8147', '0.8530', '0.8173', '0.8526', '0.8462', '0.8590', '0.8622']

['0.5497', '0.4770', '0.4119', '0.6309', '0.4720', '0.6366', '0.4516', '0.4458', '0.4243', '0.4208']


In [40]:
print(sum([float(i) for i in accuracy])/10)
print()
print(sum([float(i) for i in losss])/10)

0.8457800000000001

0.49205999999999994


In [41]:
test_X=np.array(test_sc.iloc[:,2:]).reshape(76, 600, -1)
test_X.shape

(76, 600, 18)

In [42]:
preds = []
for model in models:
    pred = model.predict(test_X)
    preds.append(pred)
pred = np.mean(preds, axis=0)
pred

array([[3.3137949e-05, 2.1677962e-01, 1.2704056e-06, ..., 1.1569775e-05,
        6.1953080e-01, 1.8673344e-07],
       [9.0124767e-04, 2.3786895e-01, 2.0319039e-05, ..., 3.2262757e-05,
        1.5562007e-01, 1.2238205e-06],
       [1.9967064e-04, 8.8423297e-02, 2.0058437e-06, ..., 3.2195945e-05,
        5.8947718e-01, 2.0121711e-06],
       ...,
       [9.4937645e-03, 5.9643318e-04, 3.5271264e-04, ..., 2.5471675e-04,
        1.0540507e-04, 1.0774012e-01],
       [2.4439324e-02, 2.7237949e-03, 8.7398061e-05, ..., 1.3677953e-03,
        8.9700734e-05, 1.7851952e-01],
       [5.2897923e-04, 1.1170584e-04, 1.1840065e-05, ..., 1.2710578e-04,
        7.3052339e-05, 4.0960473e-01]], dtype=float32)

In [44]:
submission=pd.read_csv('./sample_submission_0616Lee.csv')
submission.iloc[:,1:]=pred
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,...,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0,0.000033,0.216780,0.000001,2.304875e-07,0.005119,0.000073,0.006855,0.000006,0.000002,0.000012,0.000634,0.000060,8.701382e-09,0.000178,0.000034,0.000044,1.761358e-03,0.002399,0.000090,0.000105,0.000477,0.000158,8.523672e-09,0.000021,...,0.000123,0.013445,0.000123,0.000252,0.002353,0.000464,0.000046,0.000127,0.000562,0.001032,0.002458,0.000294,0.000169,0.000237,0.000126,2.731252e-07,5.536824e-08,1.653439e-07,0.000048,1.862531e-07,0.000053,3.193653e-10,0.000012,0.619531,1.867334e-07
1,1,0.000901,0.237869,0.000020,1.075717e-06,0.019341,0.000552,0.034963,0.001162,0.000034,0.000289,0.002232,0.000707,3.663784e-07,0.000967,0.000025,0.001595,6.480485e-03,0.011268,0.000736,0.000765,0.001681,0.001307,1.266715e-06,0.000187,...,0.000627,0.082675,0.000925,0.001401,0.022617,0.000699,0.000217,0.000303,0.011345,0.006809,0.007643,0.001229,0.003017,0.001176,0.003434,1.191768e-05,1.618906e-06,6.451095e-06,0.000448,3.590443e-06,0.000599,1.343261e-08,0.000032,0.155620,1.223821e-06
2,2,0.000200,0.088423,0.000002,1.600786e-06,0.045726,0.000392,0.003061,0.000026,0.000013,0.000102,0.001133,0.000111,5.830112e-08,0.000197,0.000079,0.000193,1.606288e-03,0.004511,0.000558,0.000292,0.000800,0.000172,2.283222e-07,0.000052,...,0.000181,0.003308,0.000478,0.000274,0.003770,0.001397,0.000110,0.000150,0.000533,0.000364,0.004485,0.000225,0.000776,0.001460,0.000953,1.114920e-06,2.855551e-07,1.426241e-06,0.000157,1.206894e-06,0.000198,1.130814e-08,0.000032,0.589477,2.012171e-06
3,3,0.004559,0.101666,0.000070,1.996433e-04,0.007421,0.006646,0.019953,0.004027,0.000023,0.000145,0.000745,0.000010,4.907999e-07,0.000035,0.000078,0.002877,1.314297e-03,0.010105,0.000484,0.001106,0.000210,0.002055,7.698406e-05,0.000167,...,0.000800,0.003985,0.001817,0.000340,0.059454,0.021787,0.000191,0.000121,0.001126,0.000269,0.002702,0.003927,0.015111,0.026035,0.135528,5.902705e-06,1.354985e-06,4.168314e-05,0.008514,1.982548e-05,0.002052,8.629020e-07,0.000129,0.148905,1.615707e-05
4,4,0.095148,0.221749,0.000108,1.437838e-04,0.000188,0.000151,0.146353,0.002076,0.000176,0.004950,0.000205,0.000372,1.059002e-04,0.000118,0.000408,0.003985,4.725608e-03,0.000518,0.000079,0.000480,0.000031,0.002248,2.609558e-05,0.000050,...,0.001638,0.031113,0.007858,0.004650,0.000627,0.000271,0.001782,0.001308,0.000464,0.003011,0.000253,0.235857,0.035470,0.009145,0.000198,1.566867e-04,6.160420e-05,3.025192e-05,0.010477,9.790144e-06,0.001494,4.047729e-06,0.000102,0.001427,3.471247e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,71,0.128587,0.092072,0.000518,1.891940e-03,0.000127,0.000317,0.000457,0.002397,0.001709,0.000016,0.000042,0.000096,2.521380e-06,0.000054,0.000034,0.447334,5.323219e-03,0.052556,0.009053,0.008714,0.000208,0.002699,3.524705e-03,0.001047,...,0.000396,0.002071,0.000097,0.000107,0.001583,0.000590,0.013130,0.004190,0.000077,0.000302,0.003471,0.000252,0.003415,0.003843,0.001328,2.950664e-04,6.880833e-07,2.672819e-04,0.000093,9.241825e-05,0.002421,9.153256e-03,0.000022,0.000798,2.973501e-03
72,72,0.170183,0.009281,0.001028,5.411435e-03,0.000059,0.001127,0.000123,0.002980,0.002122,0.000017,0.000028,0.000014,6.822567e-07,0.000008,0.000024,0.080970,4.238651e-04,0.001889,0.000649,0.000524,0.000068,0.008454,1.440839e-02,0.000307,...,0.001327,0.000360,0.000016,0.000019,0.000968,0.000465,0.001771,0.001131,0.000021,0.000061,0.000340,0.000646,0.005047,0.015650,0.005307,1.445090e-04,3.027336e-07,3.825970e-04,0.000572,5.153039e-04,0.000463,1.244407e-02,0.000011,0.000133,4.145924e-03
73,73,0.009494,0.000596,0.000353,2.172152e-03,0.000032,0.000274,0.002884,0.003924,0.011947,0.001534,0.002128,0.005968,1.357916e-03,0.000083,0.001140,0.011879,2.010970e-05,0.000204,0.000076,0.000432,0.000016,0.002016,1.45

In [45]:
submission.to_csv('./result/0616Leeresult.csv',index=False)

In [46]:
test = pd.read_csv('./result/0616Leeresult.csv')

In [49]:
df = pd.DataFrame(index=range(0,76), columns=['label', 'max'])

In [50]:
label_list = []
max_list = []
for index,row in test.iterrows(): 
    label_list.append(list(row[1:]).index(max(row[1:])))
    max_list.append(max(row[1:]))
df['label']=label_list
df['max']=max_list

In [51]:
exercise = pd.read_csv('./0616Lee.csv')
exercise=exercise.drop('Time',axis=1)
exercise=exercise.drop('acc_x',axis=1)
exercise=exercise.drop('acc_y',axis=1)
exercise=exercise.drop('acc_z',axis=1)
exercise=exercise.drop('gy_x',axis=1)
exercise=exercise.drop('gy_y',axis=1)
exercise=exercise.drop('gy_z',axis=1)
exercise = exercise.drop_duplicates()

In [53]:
label = pd.read_csv('./train_labels.csv')
label = label.drop('id',axis=1)
label = label.drop_duplicates()
label = label.sort_values('label')
merge = pd.merge(df,label,on=['label'])
merge.insert( 0, 'id', None)
merge['id'] = list(range(76))

In [54]:
merge = pd.merge(merge,exercise,on=['id'])
merge.rename(columns = {'label_desc' : '결과'}, inplace = True)
merge.rename(columns = {'exercise' : '실제 운동'}, inplace = True)

In [55]:
merge

,id,label,max,결과,실제 운동
0,0,59,0.619531,Wall Ball,Burpee
1,1,59,0.589477,Wall Ball,Burpee
2,2,25,0.260089,Medicine Ball Slam,Burpee
3,3,33,0.332503,Rowing machine,Burpee
4,4,33,0.276555,Rowing machine,Burpee
...,...,...,...,...,...
71,71,29,0.222826,Power Boat pose,Walk
72,72,29,0.222101,Power Boat pose,Walk
73,73,15,0.387852,Dynamic Stretch (at your own pace),Walk
74,74,15,0.391979,Dynamic Stretch (at your own pace),Walk


In [56]:
merge.to_csv('./result/0616Leeresult_mapping.csv',encoding='utf-8',index=False)